## Table of Contents
* [Load S3 image](#Load-S3-image)
* [Show ratio1 band](#Show-ratio1-band)
* [Load the model and make predictions on the S3 image](#Load-the-model-and-make-predictions-on-the-S3-image)
* [Make standard deviation preds with GPR model](#Make-standard-deviation-preds-with-GPR-model)
* [Save preds to `envi`](#Save-preds-to-envi)

In [1]:
from ml import c2_processing
from ml.c2_processing import load_S3_image_C2RCC
from ml import data_load_S3
from ml import models
import numpy as np
import matplotlib.pyplot as plt
import os
import joblib
from imp import reload
import warnings
warnings.filterwarnings("ignore")

/var/folders/p5/fdfmn2t168g0s422k3fb2zdw0000gn/T/ipykernel_87261/1947092208.py:9: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Load S3 image 
[Table of Contents](#Table-of-Contents)

In [2]:
##Read the images from the S3Images folder (include whole path with directory)
s3image_path = "/home/ana/Downloads/mlregocean_cdom_ar-main/mlregocean_github/S3Images/subset_0_of_S3A_OL_1_EFR____20160524T090953_20160524T091153_20170930T000736_0119_004_264______MR1_R_NT_002.data"
img = c2_processing.load_S3_image_C2RCC(image_path=s3image_path)
img.shape

FileNotFoundError: [Errno 2] No such file or directory: '/home/ana/Downloads/mlregocean_cdom_ar-main/mlregocean_github/S3Images/subset_0_of_S3A_OL_1_EFR____20160524T090953_20160524T091153_20170930T000736_0119_004_264______MR1_R_NT_002.data/rrs_1.hdr'

In [ ]:
##Plot the image to check that it was read properly
plt.imshow(img[:,:,5],vmin=0,vmax=0.01)
plt.colorbar()
plt.show()

## Load and show water mask
[Table of Contents](#Table-of-Contents)

In [ ]:
##Load the mask using the function 'cdom_processing.load_mask_C2RCC'
reload(c2_processing)
mascara = c2_processing.load_mask_C2RCC(s3image_path)
mascara = (mascara==1)
plt.imshow(mascara)
plt.show()

## Show ratio1 band
[Table of Contents](#Table-of-Contents)

In [ ]:
## Show one specific band (ratio1 in the example) with the mask on
ratio = np.ma.masked_array(img[:,:,5],mascara)
plt.imshow(ratio,vmin=0,vmax=0.01)
plt.colorbar()
plt.show()
plt.close()

## Load the model and make predictions on the S3 image
[Table of Contents](#Table-of-Contents)

In [ ]:
# Load one of the models stored in the 'models' folder; you need to give the complete name (band combiantion)
name_model = "RFR"
bands_name = "S3bands"

def load_model(name_model,bands_name):
    file_model = os.path.join(data_load_S3.PATH_TO_MODELS_C2X,
                              name_model+'_'+bands_name+'_CHL.pkl')
    return joblib.load(file_model)

regressor_sklearn = load_model(name_model,bands_name)
regressor_sklearn

In [ ]:
# Select the image to predict. band names of model and image should be the same or be correspondent
reload(c2_processing) 
image_predict = c2_processing.image_to_predict_S3_C2RCC(img,data_load_S3.bands_try_C2X[bands_name])
image_predict.shape

In [ ]:
#Optiional: number of pixels that are masked
np.sum(mascara)

In [ ]:
# Load training data to add the mean and compute the min,max range of the predicted variable
_, _, skdata_y_train, _ = data_load_S3.load_C2X()


mean_y_train = np.mean(skdata_y_train)
max_cdom = np.max(skdata_y_train)
predictions = c2_processing.predict_image(image_predict,
                                            mascara,
                                            regressor_sklearn,
                                            y_range=(0,max_cdom),
                                            y_mean=mean_y_train,step=300)
                                            
                                            
predictions.shape


In [ ]:
preds = predictions[~np.isnan(predictions)]
preds.max(), preds.min(), preds.mean()

In [ ]:
#Show the results of the prediction
plt.figure(figsize=(16,8))
plt.imshow(predictions,vmin=0,vmax=20)
plt.colorbar()
plt.show()

## Save preds to `envi`
[Table of Contents](#Table-of-Contents)

In [ ]:
#Save image in envi format (hdr, img) to be opened in SNAP
folder_save = "."
cdom_processing.write_envi(predictions, os.path.join(folder_save,
                                                     name_model+'_'+bands_name+'_CHL.hdr'))
                           
